Всякие установки:

In [0]:
from google.colab import drive
drive.mount('/content/drive')
dir_path = '/content/drive/My Drive/автобрея/'

In [0]:
!pip install pymorphy2

In [0]:
import ast

In [0]:
import pickle

In [0]:
import gensim, logging

In [0]:
import pandas as pd
import nltk
nltk.download('stopwords')
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
import re

morph = MorphAnalyzer()
token = RegexpTokenizer('\w+')
stops = set(stopwords.words('russian'))

def normalize_pm(text):
    words = [morph.parse(word)[0].normal_form for word in tokenize(text) if word]
    return words

def tokenize(text):
    return token.tokenize(text)

In [0]:
!pip install git+https://github.com/lopuhin/python-adagram.git

In [0]:
!wget https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib -d adagram.joblib

In [0]:
import adagram
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

Установки кончились!

In [0]:
## преобразуем food_seed в тот же вид, что имеют нами добавленные сиды
food_file = dir_path + 'Food_word.txt'
with open(food_file, 'r', encoding = 'utf-8') as f:
  food_seed = f.readlines()
food_seed = [line.split('        ') for line in food_seed]
food_seed = [[line[1], line[2].strip('\n')] for line in food_seed]

In [0]:
food_seed

[['вкусный', '1'],
 ['большой', '1'],
 ['прекрасный', '1'],
 ['разнообразный', '1'],
 ['единственный', '1'],
 ['достойный', '1'],
 ['странный', '1'],
 ['отличный', '1'],
 ['горячий', '1'],
 ['сытный', '1'],
 ['свежий', '1'],
 ['великолепный', '1'],
 ['интересный', '1'],
 ['различный', '1'],
 ['необычный', '1'],
 ['приятный', '1'],
 ['плохой', '0'],
 ['невкусный', '0'],
 ['посредственно', '0'],
 ['понравиться', '1']]

In [0]:
## преобразуем service_seed в тот же вид, что имеют нами добавленные сиды
service_file = dir_path + 'Service_word.txt'
with open(service_file, 'r', encoding = 'utf-8') as f:
  service_seed = f.readlines()
service_seed = [line.split('\t') for line in service_seed]
service_seed = [[line[1], line[2].strip('\n')] for line in service_seed]

In [0]:
service_seed

[['приветливый', '1'],
 ['внимательный', '1'],
 ['приятный', '1'],
 ['вежливый', '1'],
 ['хороший', '1'],
 ['ненавязчивый', '1'],
 ['доброжелательный', '1'],
 ['дружелюбный', '1'],
 ['хамоватый', '0'],
 ['отличный', '1'],
 ['милый', '1'],
 ['гостеприимный', '1'],
 ['качественный', '1'],
 ['отзывчивый', '1'],
 ['радушный', '1'],
 ['красивый', '1'],
 ['душевный', '1'],
 ['веселый', '1'],
 ['понравиться', '1'],
 ['спасибо', '1']]

In [0]:
## открываем наши добавочные фудосиды
with open('/content/drive/My Drive/автобрея/food_adagram_done.txt', 'r', encoding = 'utf-8') as f:
  food_adagram = f.readlines()
food_adagram = [x.strip('\n').split('\t') for x in food_adagram]

with open('/content/drive/My Drive/автобрея/food_wordnet_done.txt', 'r', encoding = 'utf-8') as f:
  food_wordnet = f.readlines()
food_wordnet = [x.strip('\n').split('\t') for x in food_wordnet]

In [0]:
## открываем наши добавочные сервисосиды
with open('/content/drive/My Drive/автобрея/service_adagram_done.txt', 'r', encoding = 'utf-8') as f:
  service_adagram = f.readlines()
service_adagram = [x.strip('\n').split('\t') for x in service_adagram]

with open('/content/drive/My Drive/автобрея/service_wordnet_done.txt', 'r', encoding = 'utf-8') as f:
  service_wordnet = f.readlines()
service_wordnet = [x.strip('\n').split('\t') for x in service_wordnet]

In [0]:
## объединяем изначальные сиды еды с нашими (!после превращения изначальных в тот же вид почистилось сильнее. Было 174 - стало 156)
food_dirty = food_adagram + food_wordnet + food_seed
food = set([str(line) for line in food_dirty])
food = [ast.literal_eval(x) for x in food]
len(food)

156

In [0]:
## то же для сервисных сидов
service_dirty = service_adagram + service_wordnet + service_seed
service = set([str(line) for line in service_dirty])
service = [ast.literal_eval(x) for x in service]
len(service)

248

In [0]:
## сохраняем полученные сиды еды и сервиса в отдельные файлы с помощью pickle
with open("/content/drive/My Drive/автобрея/Food_final_seeds.pickle", "wb") as f:
    pickle.dump(food, f)

with open("/content/drive/My Drive/автобрея/Service_final_seeds.pickle", "wb") as f:
    pickle.dump(service, f)

Теперь все сиды мы можем просто брать из файлов пикл.

Далее сохраняем векторы:

In [0]:
with open('/content/drive/My Drive/автобрея/topics.txt', 'r', encoding = 'utf-8') as t:
  topics = t.readlines()

In [0]:
food_topic = re.findall(r'"(\w+)"', topics[0])
service_topic = re.findall(r'"(\w+)"', topics[1])

In [0]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [0]:
amodel = gensim.models.KeyedVectors.load_word2vec_format("/content/drive/My Drive/автобрея/model.bin", binary=True)
amodel.init_sims(replace=True)

2019-12-23 16:21:29,922 : INFO : loading projection weights from /content/drive/My Drive/автобрея/model.bin
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-12-23 16:21:36,258 : INFO : loaded (248978, 300) matrix from /content/drive/My Drive/автобрея/model.bin
2019-12-23 16:21:36,259 : INFO : precomputing L2-norms of word weight vectors


In [0]:
#словарь тэгов для конвертации из формата pymorphy в w2v
tags = { 'INFN':'VERB', 'NOUN':'NOUN', 'ADJF':'ADJ', 'ADJS':'ADJ','PREP':'ADP', 'ADVB':'ADV', 'CONJ':'SCONJ', 'PRCL':'PART', 'NUMR':'NUM', 'NPRO':'PRON', 'INTJ':'INTJ', 'PRED':'ADV', 'VERB':'VERB', 'GRND':'GRND', None:'None', 'PRTF':'PRTF', 'PRTS':'PRTS'}

In [0]:
def convert_word(word):                 #в модели вордувек слова хранятся так: дом_NOUN - эта функция преобразует слово в такой формат
    aword = re.sub('ё', "е", word)
    p = morph.parse(word)[0]
    tag = tags[p.tag.POS]
    new_word = aword+'_'+tag
    return new_word

In [0]:
food_topic_vectors = [amodel.wv[convert_word(topic_word)] for topic_word in food_topic]            #вектор для топика фуд
food_topic_vector = sum(food_vectors)

In [0]:
## сохраняем вектор топиков еды в пикл файл
with open("/content/drive/My Drive/автобрея/Food_topic_vector.pickle", "wb") as f:
    pickle.dump(food_topic_vector, f)

In [0]:
service_topic_vectors = [amodel.wv[convert_word(topic_word)] for topic_word in service_topic]             #  вектор для топика сервис
service_topic_vector = sum(service_vectors)

In [0]:
## сохраняем вектор сервисных топиков в пикл файл
with open("/content/drive/My Drive/автобрея/Service_topic_vector.pickle", "wb") as f:
    pickle.dump(service_topic_vector, f)

In [0]:
## Может, еще стоит сделать и сохранить отдельно вектора для сидов...? пока хз

Теперь позитивные/негативные вектора

In [0]:
positive = []                                   #собрать все позит и негат слова
negative = [] 

for j in [food_wordnet, food_adagram, service_wordnet, service_adagram]:
  for i in j:
    if i[1] == '1':
      positive.append(i[0])
    else:
      negative.append(i[0])

In [0]:
neg_vectors = [amodel.wv[convert_word(n_word)] for n_word in negative if convert_word(n_word) in amodel.wv.vocab]             #  вектор негативных слов
neg_vector = sum(neg_vectors)

In [0]:
pos_vectors = [amodel.wv[convert_word(p_word)] for p_word in positive if convert_word(p_word) in amodel.wv.vocab]                #вектор позит слов
pos_vector = sum(pos_vectors)

In [0]:
## сохраняем их в пиклы
with open("/content/drive/My Drive/автобрея/Positive_vector.pickle", "wb") as f:
    pickle.dump(pos_vector, f)

with open("/content/drive/My Drive/автобрея/Negative_vector.pickle", "wb") as f:
    pickle.dump(neg_vector, f)